In [172]:
import pandas as pd
import numpy as np
import holidays
import math

In [173]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/final_traffic_data.csv'

df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [174]:
df = df[['speed', 'travel_time', 'status', 'data_as_of', 'link_id',
       'link_points', 'encoded_poly_line', 'encoded_poly_line_lvls', 'owner',
       'transcom_id', 'borough', 'link_name']]

In [175]:
df.columns

Index(['speed', 'travel_time', 'status', 'data_as_of', 'link_id',
       'link_points', 'encoded_poly_line', 'encoded_poly_line_lvls', 'owner',
       'transcom_id', 'borough', 'link_name'],
      dtype='object')

In [176]:
df['borough'].unique()

array(['Queens'], dtype=object)

**Filter Other Cities**

In [177]:
df = df[df['borough'] == 'Queens']

In [178]:
df['borough'].unique()

array(['Queens'], dtype=object)

**Filter Other Areas**

In [179]:
df['link_name'].value_counts()

,count
link_name,
VWE N MP4.63 (Exit 6 - Jamaica Ave) - MP6.39 (Exit 11 Jewel Ave),31527
CIP S LIE - Hempstead Tpk,31527
VWE N MP8.64 (Exit 13 Northern Blvd) - Whitestone Expwy Exit 14 (Linden Pl),31527
Laurelton Pkwy S @ SSP - Belt Pkwy W 182nd St,31527
VWE S MP8.65 (Exit 13 Northern Blvd) - MP6.39 (Exit 11 Jewel Ave),31527
Whitestone Expwy S Exit 14 (Linden Pl) - VWE S MP8.65 (Exit 13 Northern Blvd),31527
VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exit 6 Jamaica Ave),31527
CIP N Hempstead Tpk - LIE,31527
CIP S Hempstead Tpk - Laurelton Pkwy @ SSP,31527


In [180]:
chosen_link = "Belt Pkwy W 182nd St - JFK Expressway"
df = df[df['link_name'] == chosen_link]

**Investigating The Status Column**

In [181]:
df['status'].unique()

array([   0, -101])

In [182]:
df[df['status'] == -101]['speed'].value_counts()

,count
speed,
0.00,422
35.05,9
33.50,7
19.26,7
27.13,6
...,...
37.59,1
51.78,1
22.06,1


In [183]:
df[df['status'] == -101]['travel_time'].value_counts()

,count
travel_time,
0.00,422
94.33,5
61.42,5
139.08,5
69.67,5
...,...
137.67,1
118.67,1
77.58,1


**Removing Unnecessary Columns**

In [184]:
data = df.drop([ 'encoded_poly_line', 'encoded_poly_line_lvls', 'owner', 'link_id', 'transcom_id'], axis = 1)
data.head()

,speed,travel_time,status,data_as_of,link_points,borough,link_name
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway


In [185]:
new_data = data[data['status'] == -101]
new_data

,speed,travel_time,status,data_as_of,link_points,borough,link_name
1089,60.89,61.83,-101,2021-05-24 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
1128,61.07,61.57,-101,2021-05-24 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
5808,55.61,67.33,-101,2021-05-29 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
6744,59.34,63.17,-101,2021-05-30 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
7663,59.65,62.83,-101,2021-05-31 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
...,...,...,...,...,...,...,...
1194742,33.08,142.50,-101,2025-05-20 21:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
1194927,37.54,160.75,-101,2025-05-21 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
1195038,20.55,46.42,-101,2025-05-21 05:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway
1195075,29.77,132.50,-101,2025-05-21 06:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway


In [186]:
data.isna().sum()

,0
speed,0
travel_time,0
status,0
data_as_of,0
link_points,0
borough,0
link_name,0


In [187]:
print(data['status'].value_counts())
print(len(data[~((data['speed'] == 0) & (data['travel_time'] == 0)) & (data['status'] == -101)]))

status
 0      29456
-101     2070
Name: count, dtype: int64
1648


In [188]:
data.loc[(data['speed'] == 0) & (data['travel_time'] == 0), 'speed'] = np.nan
data.loc[(data['speed'].isna()) & (data['travel_time'] == 0), 'travel_time'] = np.nan

In [189]:
data.isna().sum()

,0
speed,422
travel_time,422
status,0
data_as_of,0
link_points,0
borough,0
link_name,0


**Filling Invalid Rows with Forward & Backward Filling**

In [190]:
data['speed_filled'] = data['speed'].fillna(method='ffill')
data['travel_time_filled'] = data['travel_time'].fillna(method='ffill')

data['speed'] = data['speed_filled'].fillna(method='bfill')
data['travel_time'] = data['travel_time_filled'].fillna(method='bfill')

<ipython-input-190-9c1995c98dd1>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['speed_filled'] = data['speed'].fillna(method='ffill')
<ipython-input-190-9c1995c98dd1>:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['travel_time_filled'] = data['travel_time'].fillna(method='ffill')
<ipython-input-190-9c1995c98dd1>:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['speed'] = data['speed_filled'].fillna(method='bfill')
<ipython-input-190-9c1995c98dd1>:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['travel_time'] = data['travel_time_filled'].fillna(method='bfill')


In [191]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,53.02,70.83
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,56.75,66.00
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.22,64.29
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.48,64.12
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,60.27,62.20
...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,0,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,40.90,91.83
1195741,41.52,90.83,0,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,41.52,90.83
1195778,46.86,80.17,0,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,46.86,80.17
1195815,39.97,99.83,0,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,39.97,99.83


In [192]:
print(data[data['status'] == -101][['speed', 'travel_time']])

         speed  travel_time
1089     60.89        61.83
1128     61.07        61.57
5808     55.61        67.33
6744     59.34        63.17
7663     59.65        62.83
...        ...          ...
1194742  33.08       142.50
1194927  37.54       160.75
1195038  20.55        46.42
1195075  29.77       132.50
1195112  18.53       221.58

[2070 rows x 2 columns]


In [193]:
print(len(data[(data['speed'] == 0) & (data['travel_time'] == 0)]))

0


**Creating Hour and Minute Columns**

In [194]:
data['data_as_of'] = pd.to_datetime(data['data_as_of'])

data['hour'] = data['data_as_of'].dt.hour
data['minute'] = data['data_as_of'].dt.minute

**Encoding The Hour Column**

In [195]:
data['hour_sin'] = np.sin(2 * np.pi * data['hour']/24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour']/24)

**Combining The Hour and Minute Column and Encoding It**

In [196]:
data['minute_sin'] = np.sin(2 * np.pi * data['minute']/60)
data['minute_cos'] = np.cos(2 * np.pi * data['minute']/60)

In [197]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,minute,hour_sin,hour_cos,minute_sin,minute_cos
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,53.02,70.83,0,0,0.000000,1.000000,0.0,1.0
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,56.75,66.00,1,0,0.258819,0.965926,0.0,1.0
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.22,64.29,2,0,0.500000,0.866025,0.0,1.0
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.48,64.12,3,0,0.707107,0.707107,0.0,1.0
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,60.27,62.20,4,0,0.866025,0.500000,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,0,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,40.90,91.83,23,0,-0.258819,0.965926,0.0,1.0
1195741,41.52,90.83,0,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,41.52,90.83,0,0,0.000000,1.000000,0.0,1.0
1195778,46.86,80.17,0,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,46.86,80.17,1,0,0.258819,0.965926,0.0,1.0
1195815,39.97,99.83,0,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,39.97,99.83,2,0,0.500000,0.866025,0.0,1.0


**Creating Day Name Column**

In [198]:
data['day_of_week'] = data['data_as_of'].dt.day_name()
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,minute,hour_sin,hour_cos,minute_sin,minute_cos,day_of_week
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,53.02,70.83,0,0,0.000000,1.000000,0.0,1.0,Sunday
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,56.75,66.00,1,0,0.258819,0.965926,0.0,1.0,Sunday
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.22,64.29,2,0,0.500000,0.866025,0.0,1.0,Sunday
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.48,64.12,3,0,0.707107,0.707107,0.0,1.0,Sunday
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,60.27,62.20,4,0,0.866025,0.500000,0.0,1.0,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,0,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,40.90,91.83,23,0,-0.258819,0.965926,0.0,1.0,Wednesday
1195741,41.52,90.83,0,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,41.52,90.83,0,0,0.000000,1.000000,0.0,1.0,Thursday
1195778,46.86,80.17,0,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,46.86,80.17,1,0,0.258819,0.965926,0.0,1.0,Thursday
1195815,39.97,99.83,0,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,39.97,99.83,2,0,0.500000,0.866025,0.0,1.0,Thursday


**Encoding The Days Column**

In [199]:
days = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6}
data['day_of_week_num'] = data['day_of_week'].map(days)
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,minute,hour_sin,hour_cos,minute_sin,minute_cos,day_of_week,day_of_week_num
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,53.02,70.83,0,0,0.000000,1.000000,0.0,1.0,Sunday,6
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,56.75,66.00,1,0,0.258819,0.965926,0.0,1.0,Sunday,6
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.22,64.29,2,0,0.500000,0.866025,0.0,1.0,Sunday,6
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.48,64.12,3,0,0.707107,0.707107,0.0,1.0,Sunday,6
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,60.27,62.20,4,0,0.866025,0.500000,0.0,1.0,Sunday,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,0,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,40.90,91.83,23,0,-0.258819,0.965926,0.0,1.0,Wednesday,2
1195741,41.52,90.83,0,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,41.52,90.83,0,0,0.000000,1.000000,0.0,1.0,Thursday,3
1195778,46.86,80.17,0,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,46.86,80.17,1,0,0.258819,0.965926,0.0,1.0,Thursday,3
1195815,39.97,99.83,0,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,39.97,99.83,2,0,0.500000,0.866025,0.0,1.0,Thursday,3


In [200]:
data['day_sin'] = np.sin(2 * np.pi * data['day_of_week_num'] / 7)
data['day_cos'] = np.cos(2 * np.pi * data['day_of_week_num'] / 7)

In [201]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,minute,hour_sin,hour_cos,minute_sin,minute_cos,day_of_week,day_of_week_num,day_sin,day_cos
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,53.02,70.83,0,0,0.000000,1.000000,0.0,1.0,Sunday,6,-0.781831,0.623490
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,56.75,66.00,1,0,0.258819,0.965926,0.0,1.0,Sunday,6,-0.781831,0.623490
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.22,64.29,2,0,0.500000,0.866025,0.0,1.0,Sunday,6,-0.781831,0.623490
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.48,64.12,3,0,0.707107,0.707107,0.0,1.0,Sunday,6,-0.781831,0.623490
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,60.27,62.20,4,0,0.866025,0.500000,0.0,1.0,Sunday,6,-0.781831,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,0,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,40.90,91.83,23,0,-0.258819,0.965926,0.0,1.0,Wednesday,2,0.974928,-0.222521
1195741,41.52,90.83,0,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,41.52,90.83,0,0,0.000000,1.000000,0.0,1.0,Thursday,3,0.433884,-0.900969
1195778,46.86,80.17,0,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,46.86,80.17,1,0,0.258819,0.965926,0.0,1.0,Thursday,3,0.433884,-0.900969
1195815,39.97,99.83,0,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,39.97,99.83,2,0,0.500000,0.866025,0.0,1.0,Thursday,3,0.433884,-0.900969


**Creating and Encoding The Week Column**

In [202]:
data['week_of_year'] = data['data_as_of'].dt.isocalendar().week
data['week_num_sin'] = np.sin(2 * np.pi * data['week_of_year'] / 53)
data['week_num_cos'] = np.cos(2 * np.pi * data['week_of_year'] / 53)

In [203]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,hour_cos,minute_sin,minute_cos,day_of_week,day_of_week_num,day_sin,day_cos,week_of_year,week_num_sin,week_num_cos
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,53.02,70.83,0,...,1.000000,0.0,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,56.75,66.00,1,...,0.965926,0.0,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.22,64.29,2,...,0.866025,0.0,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.48,64.12,3,...,0.707107,0.0,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,60.27,62.20,4,...,0.500000,0.0,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,0,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,40.90,91.83,23,...,0.965926,0.0,1.0,Wednesday,2,0.974928,-0.222521,21,0.6068,-0.794854
1195741,41.52,90.83,0,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,41.52,90.83,0,...,1.000000,0.0,1.0,Thursday,3,0.433884,-0.900969,21,0.6068,-0.794854
1195778,46.86,80.17,0,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,46.86,80.17,1,...,0.965926,0.0,1.0,Thursday,3,0.433884,-0.900969,21,0.6068,-0.794854
1195815,39.97,99.83,0,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,39.97,99.83,2,...,0.866025,0.0,1.0,Thursday,3,0.433884,-0.900969,21,0.6068,-0.794854


**Creating and Encoding The Month Column**

In [204]:
data['month'] = data['data_as_of'].dt.month_name()
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,minute_sin,minute_cos,day_of_week,day_of_week_num,day_sin,day_cos,week_of_year,week_num_sin,week_num_cos,month
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,53.02,70.83,0,...,0.0,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507,May
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,56.75,66.00,1,...,0.0,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507,May
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.22,64.29,2,...,0.0,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507,May
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.48,64.12,3,...,0.0,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507,May
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,60.27,62.20,4,...,0.0,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507,May
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,0,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,40.90,91.83,23,...,0.0,1.0,Wednesday,2,0.974928,-0.222521,21,0.6068,-0.794854,May
1195741,41.52,90.83,0,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,41.52,90.83,0,...,0.0,1.0,Thursday,3,0.433884,-0.900969,21,0.6068,-0.794854,May
1195778,46.86,80.17,0,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,46.86,80.17,1,...,0.0,1.0,Thursday,3,0.433884,-0.900969,21,0.6068,-0.794854,May
1195815,39.97,99.83,0,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,39.97,99.83,2,...,0.0,1.0,Thursday,3,0.433884,-0.900969,21,0.6068,-0.794854,May


In [205]:
months = {"January": 1, "February": 2, "March": 3,
            "April": 4, "May": 5, "June": 6,
            "July": 7, "August": 8, "September": 9,
            "October": 10, "November": 11, "December": 12}

data['month_num'] = data['month'].map(months)

In [206]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,minute_cos,day_of_week,day_of_week_num,day_sin,day_cos,week_of_year,week_num_sin,week_num_cos,month,month_num
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,53.02,70.83,0,...,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507,May,5
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,56.75,66.00,1,...,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507,May,5
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.22,64.29,2,...,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507,May,5
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.48,64.12,3,...,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507,May,5
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,60.27,62.20,4,...,1.0,Sunday,6,-0.781831,0.623490,20,0.696551,-0.717507,May,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,0,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,40.90,91.83,23,...,1.0,Wednesday,2,0.974928,-0.222521,21,0.6068,-0.794854,May,5
1195741,41.52,90.83,0,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,41.52,90.83,0,...,1.0,Thursday,3,0.433884,-0.900969,21,0.6068,-0.794854,May,5
1195778,46.86,80.17,0,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,46.86,80.17,1,...,1.0,Thursday,3,0.433884,-0.900969,21,0.6068,-0.794854,May,5
1195815,39.97,99.83,0,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,39.97,99.83,2,...,1.0,Thursday,3,0.433884,-0.900969,21,0.6068,-0.794854,May,5


In [207]:
data['month_sin'] = np.sin(2 * np.pi * data['month_num'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['month_num'] / 12)

In [208]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,day_of_week_num,day_sin,day_cos,week_of_year,week_num_sin,week_num_cos,month,month_num,month_sin,month_cos
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,53.02,70.83,0,...,6,-0.781831,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,56.75,66.00,1,...,6,-0.781831,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.22,64.29,2,...,6,-0.781831,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.48,64.12,3,...,6,-0.781831,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,60.27,62.20,4,...,6,-0.781831,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,0,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,40.90,91.83,23,...,2,0.974928,-0.222521,21,0.6068,-0.794854,May,5,0.5,-0.866025
1195741,41.52,90.83,0,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,41.52,90.83,0,...,3,0.433884,-0.900969,21,0.6068,-0.794854,May,5,0.5,-0.866025
1195778,46.86,80.17,0,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,46.86,80.17,1,...,3,0.433884,-0.900969,21,0.6068,-0.794854,May,5,0.5,-0.866025
1195815,39.97,99.83,0,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,39.97,99.83,2,...,3,0.433884,-0.900969,21,0.6068,-0.794854,May,5,0.5,-0.866025


**Creating a Holidays Column (Binary)**

In [209]:
us_holidays = holidays.US(years=data['data_as_of'].dt.year.unique())
us_holidays

{datetime.date(2021, 1, 1): "New Year's Day", datetime.date(2021, 12, 31): "New Year's Day (observed)", datetime.date(2021, 5, 31): 'Memorial Day', datetime.date(2021, 6, 19): 'Juneteenth National Independence Day', datetime.date(2021, 6, 18): 'Juneteenth National Independence Day (observed)', datetime.date(2021, 7, 4): 'Independence Day', datetime.date(2021, 7, 5): 'Independence Day (observed)', datetime.date(2021, 9, 6): 'Labor Day', datetime.date(2021, 11, 11): 'Veterans Day', datetime.date(2021, 11, 25): 'Thanksgiving Day', datetime.date(2021, 12, 25): 'Christmas Day', datetime.date(2021, 12, 24): 'Christmas Day (observed)', datetime.date(2021, 1, 18): 'Martin Luther King Jr. Day', datetime.date(2021, 2, 15): "Washington's Birthday", datetime.date(2021, 10, 11): 'Columbus Day', datetime.date(2022, 1, 1): "New Year's Day", datetime.date(2022, 5, 30): 'Memorial Day', datetime.date(2022, 6, 19): 'Juneteenth National Independence Day', datetime.date(2022, 6, 20): 'Juneteenth National I

In [210]:
data['is_holiday'] = [1 if x else 0 for x in data['data_as_of'].dt.date.isin(us_holidays)]

data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,day_sin,day_cos,week_of_year,week_num_sin,week_num_cos,month,month_num,month_sin,month_cos,is_holiday
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,53.02,70.83,0,...,-0.781831,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025,0
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,56.75,66.00,1,...,-0.781831,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025,0
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.22,64.29,2,...,-0.781831,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025,0
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.48,64.12,3,...,-0.781831,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025,0
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,60.27,62.20,4,...,-0.781831,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,0,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,40.90,91.83,23,...,0.974928,-0.222521,21,0.6068,-0.794854,May,5,0.5,-0.866025,0
1195741,41.52,90.83,0,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,41.52,90.83,0,...,0.433884,-0.900969,21,0.6068,-0.794854,May,5,0.5,-0.866025,0
1195778,46.86,80.17,0,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,46.86,80.17,1,...,0.433884,-0.900969,21,0.6068,-0.794854,May,5,0.5,-0.866025,0
1195815,39.97,99.83,0,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,39.97,99.83,2,...,0.433884,-0.900969,21,0.6068,-0.794854,May,5,0.5,-0.866025,0


**Creating The Non-Business Day Column**

In [211]:
data['is_non_business_day'] = [1 if x > 4 or y else 0 for x, y in zip(data['day_of_week_num'], data['is_holiday'])]
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,day_cos,week_of_year,week_num_sin,week_num_cos,month,month_num,month_sin,month_cos,is_holiday,is_non_business_day
36,53.02,70.83,0,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,53.02,70.83,0,...,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025,0,1
75,56.75,66.00,0,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,56.75,66.00,1,...,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025,0,1
114,58.22,64.29,0,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.22,64.29,2,...,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025,0,1
153,58.48,64.12,0,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,58.48,64.12,3,...,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025,0,1
192,60.27,62.20,0,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,60.27,62.20,4,...,0.623490,20,0.696551,-0.717507,May,5,0.5,-0.866025,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,0,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,40.90,91.83,23,...,-0.222521,21,0.6068,-0.794854,May,5,0.5,-0.866025,0,0
1195741,41.52,90.83,0,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,41.52,90.83,0,...,-0.900969,21,0.6068,-0.794854,May,5,0.5,-0.866025,0,0
1195778,46.86,80.17,0,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,46.86,80.17,1,...,-0.900969,21,0.6068,-0.794854,May,5,0.5,-0.866025,0,0
1195815,39.97,99.83,0,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,39.97,99.83,2,...,-0.900969,21,0.6068,-0.794854,May,5,0.5,-0.866025,0,0


In [212]:
# data['is_school_day'] --> based on the region we choose

**Dropping Unnecessary Columns**

In [213]:
data.columns

Index(['speed', 'travel_time', 'status', 'data_as_of', 'link_points',
       'borough', 'link_name', 'speed_filled', 'travel_time_filled', 'hour',
       'minute', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos',
       'day_of_week', 'day_of_week_num', 'day_sin', 'day_cos', 'week_of_year',
       'week_num_sin', 'week_num_cos', 'month', 'month_num', 'month_sin',
       'month_cos', 'is_holiday', 'is_non_business_day'],
      dtype='object')

In [214]:
data = data.drop(['speed_filled', 'travel_time_filled', 'status', 'hour', 'minute', 'day_of_week', 'day_of_week_num', 'week_of_year', 'month', 'month_num'], axis = 1)
data.columns

Index(['speed', 'travel_time', 'data_as_of', 'link_points', 'borough',
       'link_name', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos',
       'day_sin', 'day_cos', 'week_num_sin', 'week_num_cos', 'month_sin',
       'month_cos', 'is_holiday', 'is_non_business_day'],
      dtype='object')

In [215]:
data

,speed,travel_time,data_as_of,link_points,borough,link_name,hour_sin,hour_cos,minute_sin,minute_cos,day_sin,day_cos,week_num_sin,week_num_cos,month_sin,month_cos,is_holiday,is_non_business_day
36,53.02,70.83,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.000000,1.000000,0.0,1.0,-0.781831,0.623490,0.696551,-0.717507,0.5,-0.866025,0,1
75,56.75,66.00,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.258819,0.965926,0.0,1.0,-0.781831,0.623490,0.696551,-0.717507,0.5,-0.866025,0,1
114,58.22,64.29,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.500000,0.866025,0.0,1.0,-0.781831,0.623490,0.696551,-0.717507,0.5,-0.866025,0,1
153,58.48,64.12,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.707107,0.707107,0.0,1.0,-0.781831,0.623490,0.696551,-0.717507,0.5,-0.866025,0,1
192,60.27,62.20,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.866025,0.500000,0.0,1.0,-0.781831,0.623490,0.696551,-0.717507,0.5,-0.866025,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,-0.258819,0.965926,0.0,1.0,0.974928,-0.222521,0.6068,-0.794854,0.5,-0.866025,0,0
1195741,41.52,90.83,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.000000,1.000000,0.0,1.0,0.433884,-0.900969,0.6068,-0.794854,0.5,-0.866025,0,0
1195778,46.86,80.17,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.258819,0.965926,0.0,1.0,0.433884,-0.900969,0.6068,-0.794854,0.5,-0.866025,0,0
1195815,39.97,99.83,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.500000,0.866025,0.0,1.0,0.433884,-0.900969,0.6068,-0.794854,0.5,-0.866025,0,0


**Choosing A Suitable Borough**

In [216]:
data['borough'].unique()

array(['Queens'], dtype=object)

In [217]:
boroughs = data['borough'].unique()

for borough in boroughs:
  borough_data = data[data['borough'] == borough]
  print("The average speed in", borough, "=", borough_data['speed'].mean())

The average speed in Queens = 37.203088879020484


In [218]:
boroughs = df['borough'].unique()

for borough in boroughs:
  borough_data = df[df['borough'] == borough]
  print("The average speed in", borough, "=", borough_data['speed'].mean())

The average speed in Queens = 36.92705512909979


In [219]:
boroughs = data['borough'].unique()

for borough in boroughs:
  borough_data = data[data['borough'] == borough]
  print("The average travel time in", borough, "=", borough_data['travel_time'].mean())

The average travel time in Queens = 117.4600913531688


In [220]:
boroughs = df['borough'].unique()

for borough in boroughs:
  borough_data = df[df['borough'] == borough]
  print("The average travel time in", borough, "=", borough_data['travel_time'].mean())

The average travel time in Queens = 116.57153682674617


In [221]:
data.columns

Index(['speed', 'travel_time', 'data_as_of', 'link_points', 'borough',
       'link_name', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos',
       'day_sin', 'day_cos', 'week_num_sin', 'week_num_cos', 'month_sin',
       'month_cos', 'is_holiday', 'is_non_business_day'],
      dtype='object')

In [222]:
data

,speed,travel_time,data_as_of,link_points,borough,link_name,hour_sin,hour_cos,minute_sin,minute_cos,day_sin,day_cos,week_num_sin,week_num_cos,month_sin,month_cos,is_holiday,is_non_business_day
36,53.02,70.83,2021-05-23 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.000000,1.000000,0.0,1.0,-0.781831,0.623490,0.696551,-0.717507,0.5,-0.866025,0,1
75,56.75,66.00,2021-05-23 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.258819,0.965926,0.0,1.0,-0.781831,0.623490,0.696551,-0.717507,0.5,-0.866025,0,1
114,58.22,64.29,2021-05-23 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.500000,0.866025,0.0,1.0,-0.781831,0.623490,0.696551,-0.717507,0.5,-0.866025,0,1
153,58.48,64.12,2021-05-23 03:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.707107,0.707107,0.0,1.0,-0.781831,0.623490,0.696551,-0.717507,0.5,-0.866025,0,1
192,60.27,62.20,2021-05-23 04:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.866025,0.500000,0.0,1.0,-0.781831,0.623490,0.696551,-0.717507,0.5,-0.866025,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195704,40.90,91.83,2025-05-21 23:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,-0.258819,0.965926,0.0,1.0,0.974928,-0.222521,0.6068,-0.794854,0.5,-0.866025,0,0
1195741,41.52,90.83,2025-05-22 00:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.000000,1.000000,0.0,1.0,0.433884,-0.900969,0.6068,-0.794854,0.5,-0.866025,0,0
1195778,46.86,80.17,2025-05-22 01:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.258819,0.965926,0.0,1.0,0.433884,-0.900969,0.6068,-0.794854,0.5,-0.866025,0,0
1195815,39.97,99.83,2025-05-22 02:00:00,"40.6665206,-73.76246 40.66738,-73.77021 40.667...",Queens,Belt Pkwy W 182nd St - JFK Expressway,0.500000,0.866025,0.0,1.0,0.433884,-0.900969,0.6068,-0.794854,0.5,-0.866025,0,0


In [223]:
data.to_csv('/content/drive/My Drive/preprocessed_data.csv', index=False)

In [228]:
print(f"Max Speed: {data['speed'].max()}. Min Speed: {data['speed'].min()}")

Max Speed: 63.01. Min Speed: 0.31


In [227]:
print(f"Max Time: {data['travel_time'].max()}. Min Time: {data['travel_time'].min()}")

Max Time: 1040.92. Min Time: 5.25
